<a href="https://colab.research.google.com/github/ZKTKZ/thdne/blob/master/StyleGAN2_Tazik_25GB_RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab Hacks

Induce Colab RAM -> 25gb


```
i = []
while True:
  i.append(i)
```

Keep Colab from disconnecting after 1.5hrs.

```

function KeepClicking(){
   console.log("Clicking");
   document.querySelector("colab-toolbar-button#connect").click()
}setInterval(KeepClicking,60000)
```







In [2]:
from google.colab import drive
drive.mount('/content/drive')

%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/ZKTKZ/stylegan2.git
%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

!pip install tensorboard

url = 'https://drive.google.com/open?id=1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez'
#'https://drive.google.com/open?id=1BHeqOZ58WZ-vACR2MJkh1ZVbJK2B-Kle'
model_id = url.replace('https://drive.google.com/open?id=', '')

!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

network_pkl = './models/model_%s.pkl' % model_id#(hashlib.md5(model_id.encode()).hexdigest())
gdd.download_file_from_google_drive(file_id=model_id,
                                    dest_path=network_pkl)

!python dataset_tool.py create_from_images ./dataset/hayasaka /content/drive/'My Drive'/twist_moe/hand_tuned_larger_2x/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'stylegan2'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 719 (delta 0), reused 0 (delta 0), pack-reused 716
Receiving objects: 100% (719/719), 16.77 MiB | 39.66 MiB/s, done.
Resolving deltas: 100% (463/463), done.
/content/stylegan2/stylegan2
CPU says hello.
GPU says hello.
Tensorflow version: 1.15.2
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ec33b791-3b95-7913-c80a-4dfe26657b8a)
GPU Identified at: /device:GPU:0
Loading images from "/content/drive/My Drive/twist_moe/hand_tuned_larger_2x/"
detected 413 images ...
Creating dataset "./dataset/hayasaka"
Adding the images to tfrecords ...
Added 413 images.


In [ ]:
!python run_training.py --spatial-augmentations=true --lr=0.0005 --num-gpus=1 --data-dir=./dataset --config=config-f --dataset=hayasaka --mirror-augment=true --metric=none --total-kimg=10000 --min-h=4 --min-w=4 --res-log2=7 --result-dir="/content/drive/My Drive/twist_moe/results/" --resume-pkl='./models/model_1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez.pkl'
'''
#i won't have time to ACTUALLY think about research for at least a week; so I'm just gonna keep this running w/o really knowing what to expect
# not very clean, i know, but better than wasting those potential colab hours. 
'''

Local submit - run_dir: /content/drive/My Drive/twist_moe/results/00001-stylegan2-hayasaka-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
Loading networks from "./models/model_1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.

G                             Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
lod                           -         ()                  -               
dlatent_avg                   -       

In [ ]:

!python run_generator.py generate-images --seeds=0-50 --truncation-psi=1.0 --network=/content/drive/'My Drive'/twist_moe/results/00007-stylegan2-hayasaka-1gpu-config-f/network-snapshot-000086.pkl
%cp -av /content/stylegan2/results/00000-generate-images /content/drive/'My Drive'/twist_moe/seeds-1.0


In [ ]:
import math
import moviepy.editor
from numpy import linalg
import numpy as np
import pickle

def main():
    tflib.init_tf()
    _G, _D, Gs = pickle.load(open("/content/drive/'My Drive'/twist_moe/results/00006-stylegan2-hayasaka-1gpu-config-f/network-snapshot-000123.pkl", "rb"))

    rnd = np.random
    latents_a = rnd.randn(1, Gs.input_shape[1])
    latents_b = rnd.randn(1, Gs.input_shape[1])
    latents_c = rnd.randn(1, Gs.input_shape[1])

    def circ_generator(latents_interpolate):
        radius = 40.0

        latents_axis_x = (latents_a - latents_b).flatten() / linalg.norm(latents_a - latents_b)
        latents_axis_y = (latents_a - latents_c).flatten() / linalg.norm(latents_a - latents_c)

        latents_x = math.sin(math.pi * 2.0 * latents_interpolate) * radius
        latents_y = math.cos(math.pi * 2.0 * latents_interpolate) * radius

        latents = latents_a + latents_x * latents_axis_x + latents_y * latents_axis_y
        return latents

    def mse(x, y):
        return (np.square(x - y)).mean()

    def generate_from_generator_adaptive(gen_func):
        max_step = 1.0
        current_pos = 0.0

        change_min = 10.0
        change_max = 11.0

        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

        current_latent = gen_func(current_pos)
        current_image = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
        array_list = []

        video_length = 1.0
        while(current_pos < video_length):
            array_list.append(current_image)

            lower = current_pos
            upper = current_pos + max_step
            current_pos = (upper + lower) / 2.0

            current_latent = gen_func(current_pos)
            current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
            current_mse = mse(array_list[-1], current_image)

            while current_mse < change_min or current_mse > change_max:
                if current_mse < change_min:
                    lower = current_pos
                    current_pos = (upper + lower) / 2.0

                if current_mse > change_max:
                    upper = current_pos
                    current_pos = (upper + lower) / 2.0


                current_latent = gen_func(current_pos)
                current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
                current_mse = mse(array_list[-1], current_image)
            print(current_pos, current_mse)
        return array_list

    frames = generate_from_generator_adaptive(circ_generator)
    frames = moviepy.editor.ImageSequenceClip(frames, fps=30)

    # Generate video.
    mp4_file = 'results/circular.mp4'
    mp4_codec = 'libx264'
    mp4_bitrate = '3M'
    mp4_fps = 20

    frames.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

if __name__ == "__main__":
    main()